In [ ]:
import os
import sys
import yaml
import argparse
from glob import glob
from datetime import datetime, timedelta

import numpy as np
import xarray as xr

In [ ]:
sys.path.insert(0, os.path.realpath('../libs/'))
import verif_utils as vu

In [ ]:
config_name = os.path.realpath('verif_config.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

In [ ]:
model_names = ['wxformer', 'IFS']
VERIF = {}

# file name indices
IND_max = 2192
INDs = np.arange(0, IND_max+40, 40)
INDs[-1] = IND_max

In [ ]:
for model_name in model_names:
    # file names to load
    verif_lead_range = conf[model_name]['verif_lead_range']
    path_ACC_verif = conf[model_name]['save_loc_verif']+'combined_acc_{:04d}_{:04d}_{:03d}h_{:03d}h_{}.nc'
    path_RMSE_verif = conf[model_name]['save_loc_verif']+'combined_rmse_{:04d}_{:04d}_{:03d}h_{:03d}h_{}.nc'

    # file names to save
    path_ACC_save = conf[model_name]['save_loc_verif']+'ACC_{:03d}h_{:03d}h_{}.nc'
    path_RMSE_save = conf[model_name]['save_loc_verif']+'RMSE_{:03d}h_{:03d}h_{}.nc'

    # load xarray.Dataset and merge all verified days
    ACC_verif = []
    RMSE_verif = []

    for i, ind_start in enumerate(INDs[:-1]):
        ind_end = INDs[i+1]
        filename = path_ACC_verif.format(ind_start, ind_end, verif_lead_range[0], verif_lead_range[-1], model_name)
        ds_verf_temp = xr.open_dataset(filename)
        ACC_verif.append(ds_verf_temp)
    
        filename = path_RMSE_verif.format(ind_start, ind_end, verif_lead_range[0], verif_lead_range[-1], model_name)
        ds_verf_temp = xr.open_dataset(filename)
        RMSE_verif.append(ds_verf_temp)

    # merge by concat
    ds_ACC_verif = xr.concat(ACC_verif, dim='days')
    ds_RMSE_verif = xr.concat(RMSE_verif, dim='days')

    # save to one dictionary for some checking
    VERIF['{}_ACC'.format(model_name)] = ds_ACC_verif
    VERIF['{}_RMSE'.format(model_name)] = ds_RMSE_verif

    # save to nc
    save_name_ACC = path_ACC_save.format(verif_lead_range[0], verif_lead_range[-1], model_name)
    ds_ACC_verif.to_netcdf(save_name_ACC)
    print('Save to {}'.format(save_name_ACC))
    
    save_name_RMSE = path_RMSE_save.format(verif_lead_range[0], verif_lead_range[-1], model_name)
    ds_ACC_verif.to_netcdf(save_name_RMSE)
    print('Save to {}'.format(save_name_RMSE))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
np.mean(np.array(VERIF['IFS_ACC']['t2m']), axis=0)

In [ ]:
VERIF['wxformer_ACC']

In [ ]:
test = np.mean(np.array(VERIF['wxformer_ACC']['t2m']), axis=1)

In [ ]:
np.argwhere(np.isnan(test))

In [ ]:
VERIF['wxformer_ACC'].isel(days=640)

In [ ]:
VERIF['wxformer_ACC'].hour[640]

In [ ]:
plt.plot(np.mean(np.array(VERIF['IFS_ACC']['t2m']), axis=0))
plt.plot(np.nanmean(np.array(VERIF['wxformer_ACC']['t2m']), axis=0))

In [ ]:
np.array(VERIF['wxformer_ACC']['t2m'])

In [ ]:
np.array(VERIF['IFS_ACC']['t2m'])